In [ ]:
from PIL import Image
import torch
from torch.utils import data 
import numpy as np
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm

In [ ]:
train_transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5,0.5,0.5],
                                    std=[0.5,0.5,0.5]),
])

test_transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5,0.5,0.5],
                                    std=[0.5,0.5,0.5]),
])

verif_transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5,0.5,0.5],
                                    std=[0.5,0.5,0.5]),
])

In [ ]:
train_dataset=torchvision.datasets.ImageFolder(
  'D:/train',
    transform=train_transformer
)

valid_dataset=torchvision.datasets.ImageFolder(
  'D:/valid',
   transform=valid_transformer
)

test_dataset=torchvision.datasets.ImageFolder(
  'D:/test',
   transform=test_transformer
) 

In [ ]:
Batch_size=16
dl_train=torch.utils.data.DataLoader(
        train_dataset,
        batch_size=Batch_size,
        shuffle=True
)
dl_valid=torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=Batch_size,
        shuffle=True
)
dl_test=torch.utils.data.DataLoader(
        test_dataset,
        batch_size=Batch_size,
        shuffle=True
)

In [ ]:
####Efficient-net
from efficientnet_pytorch import EfficientNet
####ensemble
class CombinedModel(nn.Module):  
    ''' Ensemble deep learning model '''
    def __init__(self, num_classes):  
        super(CombinedModel, self).__init__()  
        
        # ResNet Backbone  
        self.resnet = torchvision.models.resnet18(pretrained=True)  
        num_ftrs_resnet = self.resnet.fc.in_features  
        self.resnet.fc = nn.Linear(num_ftrs_resnet, num_classes) 
        
        # EfficientNet Backbone  
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b7')
        num_ftrs_efficientnet = self.efficientnet._fc.in_features  
        self.efficientnet._fc = nn.Linear(num_ftrs_efficientnet, num_classes)  
        
        # Densenet Backbone
        self.densenet = torchvision.models.densenet121(pretrained=True)  # 使用预训练的 DenseNet  
        num_ftrs_densenet = self.densenet.classifier.in_features  
        self.densenet.classifier = nn.Linear(num_ftrs_densenet, num_classes)
        
        #self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):  
        result_resnet = self.resnet(x)
        result_efficientnet = self.efficientnet(x)
        result_densenet = self.densenet(x)
        #result_resnet = self.Softmax(self.resnet(x))
        #result_efficientnet = self.Softmax(self.efficientnet(x))
        #result_densenet = self.Softmax(self.densenet(x))
        
        output = (result_resnet + result_densenet + result_efficientnet ) / 3
        return output  

In [ ]:
num_classes = 2  
model = CombinedModel(num_classes) 
model

In [ ]:
###LOSS
loss_fn=nn.CrossEntropyLoss() 
###Optimizer and learning rate
from torch.optim import lr_scheduler
optim=torch.optim.Adam(model.parameters(),lr=0.000002)
scheduler = lr_scheduler.StepLR(optim,step_size=7,gamma = 0.99)

In [ ]:
if torch.cuda.is_available():
    model.to('cuda')
torch.cuda.is_available()

In [ ]:
def fit(epoch, model, trainloader, validloader, testloader):
    correct = 0
    total = 0
    running_loss = 0
    model.train()
    for x, y in tqdm(trainloader):
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
        
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total
        
        
    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0 
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(validloader):
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()
    
    epoch_valid_loss = valid_running_loss / len(validloader.dataset)
    epoch_valid_acc = valid_correct / valid_total
    
    static_dict=model.state_dict()
    torch.save(static_dict,'./checkpoints/{}_train_acc_{}_valid_acc_{}.pth'.format(epoch,round(epoch_acc, 3),round(epoch_valid_acc,3)))
        
    print('epoch: ', epoch, 
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'valid_loss： ', round(epoch_valid_loss, 3),
          'valid_accuracy:', round(epoch_valid_acc, 3)
             )
        
    return epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc

In [ ]:
epochs = 25

In [ ]:
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []


for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc  = fit(epoch,
                                                                    model,
                                                                    dl_train,
                                                                    dl_valid)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)

In [ ]:
####loss curve
plt.plot(range(1, epochs+1), train_loss, label='train set', color='#0000FF')
plt.plot(range(1, epochs+1), valid_loss, label='validation set', color='#FF0000')
plt.title('Model loss function convergence curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
###ACC
plt.plot(range(1, epochs+1), train_acc, label='train set', color='#0000FF')
plt.plot(range(1, epochs+1), valid_acc, label='validation set', color='#FF0000')
plt.title('The accuracy of different deep learning nets in the training and validation set changes with epochs')
plt.xlabel('Epoch')
plt.ylabel('Accucary')
plt.legend()

In [ ]:
from sklearn.metrics import confusion_matrix
model.train()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in dl_train:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        
        outputs = model(inputs)
        
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

###confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print(cm) 

In [ ]:
model.eval()
all_preds1 = []
all_labels1 = []

with torch.no_grad():
    for inputs, labels in dl_valid:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        
        # 前向传播
        outputs = model(inputs)
        
        # 获取预测结果
        _, preds = torch.max(outputs, 1)
        
        # 收集预测值和真实标签
        all_preds1.extend(preds.cpu().numpy())
        all_labels1.extend(labels.cpu().numpy())
###confusion matrix1
cm1 = confusion_matrix(all_labels1, all_preds1)
print(cm1) 

In [ ]:
model.eval()
all_preds2 = []
all_labels2 = []

with torch.no_grad():
    for inputs, labels in dl_test:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        
        # 前向传播
        outputs = model(inputs)
        
        # 获取预测结果
        _, preds = torch.max(outputs, 1)
        
        # 收集预测值和真实标签
        all_preds2.extend(preds.cpu().numpy())
        all_labels2.extend(labels.cpu().numpy())

###confusion matrix2
cm2 = confusion_matrix(all_labels2, all_preds2)
print(cm2) 

In [ ]:
import statsmodels.api as sm
from sklearn.utils import resample
TN = 35  # True Negatives
TP = 31  # True Positives
FP = 13 # False Positives
FN = 17  # False Negatives

# 
accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
NPV = TN / (TN + FN) if (TN + FN) > 0 else 0
PPV = TP / (TP + FP) if (TP + FP) > 0 else 0

# 95%CI
def calc_confidence_interval(successes, trials, confidence=0.95):
    ci = sm.stats.proportion_confint(successes, trials, alpha=1-confidence, method='normal')
    return ci
#
accuracy_ci = calc_confidence_interval(TP + TN, TP + TN + FP + FN)
sensitivity_ci = calc_confidence_interval(TP, TP + FN)
specificity_ci = calc_confidence_interval(TN, TN + FP)
NPV_ci = calc_confidence_interval(TN, TN + FN)
PPV_ci = calc_confidence_interval(TP, TP + FP)

# 
print("Accuracy: {:.3f} (95% CI: [{:.3f}, {:.3f}])".format(accuracy, accuracy_ci[0], accuracy_ci[1]))
print("Sensitivity: {:.3f} (95% CI: [{:.3f}, {:.3f}])".format(sensitivity, sensitivity_ci[0], sensitivity_ci[1]))
print("Specificity: {:.3f} (95% CI: [{:.3f}, {:.3f}])".format(specificity, specificity_ci[0], specificity_ci[1]))
print("NPV: {:.3f} (95% CI: [{:.3f}, {:.3f}])".format(NPV, NPV_ci[0], NPV_ci[1]))
print("PPV: {:.3f} (95% CI: [{:.3f}, {:.3f}])".format(PPV, PPV_ci[0], PPV_ci[1]))

In [ ]:
torch.save(model,'EDLM.pkl')